In [1]:
import torch
import numpy as np
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import to_hex
from places import generate_places

In [2]:
save_path = f"./figures/"
network_type = "all"
place_name1 = "huangpu" # huangpu westminster
place_name2 = "pasadena" # pasadena paris_centre

data_dir1, places1 = generate_places(place_name1)
data_dir2, places2 = generate_places(place_name2)

In [3]:
G1 = ox.graph_from_place(places1, network_type=network_type, retain_all=False)
G2 = ox.graph_from_place(places2, network_type=network_type, retain_all=False)
len(list(G1.nodes)), len(list(G2.nodes))

(5292, 12420)

In [4]:
labels1 = torch.load(f'{data_dir1}10-chunk_node_labels.pt', weights_only=True).numpy()
labels2 = torch.load(f'{data_dir2}10-chunk_node_labels.pt', weigts_only=True).numpy()

FileNotFoundError: [Errno 2] No such file or directory: './road_data_2025/huangpu/10-chunk_node_labels.pt'

In [ ]:
def get_colors(labels):
    # cmap = sns.color_palette("Blues_r", as_cmap=True)
    cmap = sns.color_palette("rocket", as_cmap=True)
    palette = cmap(np.linspace(0.1, 0.85, 10))
    palette_hex = [to_hex(color) for color in palette]
    color_array = [palette_hex[index] for index in labels]
    return color_array

In [ ]:
my_font = 23
node_size = 20
fig, ax = plt.subplots(1,2, figsize=(24, 7), gridspec_kw={'width_ratios': [1, 1]})
ox.plot_graph(
    G1, ax=ax[0], node_size=node_size+10, node_color=get_colors(labels1), node_alpha=0.5,
    edge_linewidth=1, edge_color='black', edge_alpha=0.5, show=False, 
)
ox.plot_graph(
    G2, ax=ax[1], node_size=node_size, node_color=get_colors(labels2), node_alpha=0.5,
    edge_linewidth=1, edge_color='black', edge_alpha=0.5, show=False, 
)

ax[0].set_title(f"(a) Huangpu, Shanghai", fontsize=my_font)
ax[1].set_title(f"(b) Pasadena, Los Angeles", fontsize=my_font)


plt.tight_layout(pad=10.0)
plt.savefig(f"./label_visual_map_{place_name1}_and_{place_name2}.jpg")
plt.show()

In [ ]:
dataset_name = "paris"
dataset_path = f'./data/{dataset_name}'


d_labels = {
    "paris":"Paris", "shanghai":"Shanghai", "la":"L.A.", "london":"London",
}

d_color = {
    "paris":"#c5373e", # Red, "#9c251c", rgb(197, 55, 62)
    "shanghai":   "#006eae", # Blue, "#00498d", rgb(0, 110, 174)
    "la":  "#439130", # Green, "#1c6e2b",   rgb(67, 145, 48)
    "london":   "#6e788e", # Grey # "#43536a",   rgb(110, 120, 142)
}

d_marker = {"paris":'x', "shanghai":'o', "la":'^', "london":"s"}


eccentricities = {}
for dataset_name in ["paris", "shanghai", "la", "london"]:
    dataset_path = f'./data/{dataset_name}'
    eccentricities[dataset_name] = torch.load(f'{dataset_path}/16-hop_eccentricities.pt')

In [ ]:
num_bins = 100
fontsize = 26

# Create the figure and axis
fig, ax = plt.subplots(1, 1, figsize=(11, 8))

# Plot histograms for each dataset on the same subplot
for dataset_name in ["paris", "shanghai", "la", "london"]:
    ax.hist(
        x=eccentricities[dataset_name]/1000,
        bins=num_bins,
        range=(0, 10),  # Limit the x-axis to focus on the main distribution

        density=True,
        alpha=0.7,
        label=d_labels[dataset_name],
        color=d_color[dataset_name],
        edgecolor='black'
    )


# Set labels and title
ax.set_xlabel("16-hop eccentricity (km)", fontsize=fontsize)
ax.set_ylabel("Density", fontsize=fontsize)
ax.set_title("Distributions of node eccentricity estimations", fontsize=fontsize + 2)
ax.tick_params(axis='both', labelsize=fontsize - 3)
# ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))

# Add gridlines for readability
ax.grid(visible=True, linestyle='--', alpha=0.6)

# Add legend
ax.legend(fontsize=fontsize - 5, frameon=True)

# Apply custom styling and adjust layout
plt.tight_layout()
plt.savefig(f"./label_eccentricity_distributions.jpg", bbox_inches='tight')
plt.show()